Scrape the Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [1]:
import pandas as pd

tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

Postal Codes are in the first table

In [2]:
postal_codes = tables[0]
postal_codes.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Filter dataframe deleting rows containing Not assigned Postal Codes. Rename first column.

In [3]:
postal_codes = postal_codes[postal_codes['Borough'] != 'Not assigned'].reset_index(drop=True)

postal_codes = postal_codes.rename(columns={'Postal code': 'PostalCode'})

postal_codes.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In order to check duplicate postal codes we can count unique items for each column.

In [4]:
postal_codes.describe()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,10,98
top,M1W,North York,Downsview
freq,1,24,4


No duplicate postal codes! Neighborhoods are already grouped by postal code.
Let's check if there is any NaN or Not Assigned in Neighborhood column.

In [5]:
postal_codes['Neighborhood'].isna().sum()

0

In [6]:
(postal_codes['Neighborhood']=='Not assigned').sum()

0

Multiple Neighborhoods for the same Postal Code are formatted with '/', replace it with a single comma (,).

In [7]:
postal_codes['Neighborhood'].replace(' /',',',regex=True,inplace=True)
postal_codes.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
postal_codes.shape

(103, 3)